In [34]:
import os
import gmaps
import gmaps.datasets
import config
# Try to move it to environement variables.
gmaps.configure(api_key=config.api)

In [36]:
print(os.environ["PATH"])

/Library/Frameworks/Python.framework/Versions/3.5/bin:/Users/pradeepsaiuppula/.rvm/gems/ruby-2.4.0/bin:/Users/pradeepsaiuppula/.rvm/gems/ruby-2.4.0@global/bin:/Users/pradeepsaiuppula/.rvm/rubies/ruby-2.4.0/bin:/Users/pradeepsaiuppula/bin:/usr/local/bin:/Library/Frameworks/Python.framework/Versions/3.5/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/pradeepsaiuppula/.local/bin:/Users/pradeepsaiuppula/.rvm/bin


In [27]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig